In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import tensorflow
import keras
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.layers as layers
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input

2023-05-02 14:50:12.499142: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/jiehyun.kim001/miniconda3/envs/O/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import cactas as C

In [4]:
asymp_img = np.load('/raid/mpsych/CACTAS/DATA/ESUS/asymp_cropped_no_norm.npy')
symp_img = np.load('/raid/mpsych/CACTAS/DATA/ESUS/symp_cropped_no_norm.npy')
asymp_label = np.load('/raid/mpsych/CACTAS/DATA/ESUS/asymp_cropped_labels_no_norm.npy')
symp_label = np.load('/raid/mpsych/CACTAS/DATA/ESUS/symp_cropped_labels_no_norm.npy')

In [5]:
images = np.concatenate((asymp_img, symp_img, asymp_label, symp_label))
#images = np.concatenate((asymp_img, asymp_label, symp_img, symp_label))
labels = np.array([0]*len(asymp_img) + [0]*len(asymp_label) + [1]*len(symp_img) + [1]*len(symp_label))

In [6]:
images.shape

(2834, 28, 28, 1)

In [7]:
labels.shape

(2834,)

In [8]:
X, y = shuffle(images, labels, random_state=0)

In [9]:
X_train, X_test, y_train, y_test = C.Util.split_3(X, y, val_size=0.15)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2411, 28, 28, 1), (423, 28, 28, 1), (2411,), (423,))

In [11]:
train_data_gen = ImageDataGenerator(
        rotation_range=5.,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=40,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='constant'
    )

In [12]:
test_data_gen = ImageDataGenerator()

In [13]:
train = train_data_gen.flow(X_train, y_train)

In [14]:
test = test_data_gen.flow(X_test, y_test)

In [15]:
'''
model = tensorflow.keras.Sequential()
model.add(layers.Flatten(input_shape=(28, 28, 1)))
model.add(layers.Dense(units=256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(units=128, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(units=1, activation='sigmoid'))
'''

"\nmodel = tensorflow.keras.Sequential()\nmodel.add(layers.Flatten(input_shape=(28, 28, 1)))\nmodel.add(layers.Dense(units=256, activation='relu'))\nmodel.add(layers.Dropout(0.5))\nmodel.add(layers.Dense(units=128, activation='relu'))\nmodel.add(layers.Dropout(0.3))\nmodel.add(layers.Dense(units=1, activation='sigmoid'))\n"

In [16]:
#CNN
model = tensorflow.keras.Sequential()

model.add(layers.Conv2D(filters=20, kernel_size=(5, 5), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(filters=50, kernel_size=(5, 5), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())

model.add(layers.Dense(units=256, activation='relu'))

model.add(layers.Dropout(0.5))

model.add(layers.Dense(units=1, activation = 'softmax'))  


2023-05-02 14:50:13.878803: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-05-02 14:50:14.177748: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:07:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-05-02 14:50:14.180354: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:87:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-05-02 14:50:14.180373: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-05-02 14:50:14.183720: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-05-02 

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 20)        520       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 20)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 50)          25050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 50)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0

In [18]:
sgd = tensorflow.keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

/home/jiehyun.kim001/miniconda3/envs/O/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [19]:
model.compile(loss=tensorflow.keras.losses.binary_crossentropy,
              #loss='sparse_categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [20]:
batch_size = 32

In [21]:
model.fit(train, steps_per_epoch=len(train) // batch_size, verbose=True, epochs=100)

2023-05-02 14:50:15.818395: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-05-02 14:50:15.837172: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2245645000 Hz


Epoch 1/100


2023-05-02 14:50:16.219630: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-05-02 14:50:17.072563: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2023-05-02 14:50:17.703429: W tensorflow/stream_executor/gpu/asm_compiler.cc:191] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.0
2023-05-02 14:50:17.703456: W tensorflow/stream_executor/gpu/asm_compiler.cc:194] Used ptxas at ptxas
2023-05-02 14:50:17.703522: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-05-02 14:50:17.723474: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-05-02 14:50:18.442610: I tensorflow/stream_exec

2/2 [==============================] - 3s 10ms/step - loss: 0.7034 - accuracy: 0.4375
Epoch 2/100
2/2 [==============================] - 0s 15ms/step - loss: 0.7178 - accuracy: 0.3906
Epoch 3/100
2/2 [==============================] - 0s 15ms/step - loss: 0.7196 - accuracy: 0.4219
Epoch 4/100
2/2 [==============================] - 0s 15ms/step - loss: 0.7022 - accuracy: 0.6094
Epoch 5/100
2/2 [==============================] - 0s 14ms/step - loss: 0.7117 - accuracy: 0.5469
Epoch 6/100
2/2 [==============================] - 0s 15ms/step - loss: 0.6969 - accuracy: 0.5000
Epoch 7/100
2/2 [==============================] - 0s 14ms/step - loss: 0.6872 - accuracy: 0.3750
Epoch 8/100
2/2 [==============================] - 0s 14ms/step - loss: 0.6725 - accuracy: 0.3906
Epoch 9/100
2/2 [==============================] - 0s 14ms/step - loss: 0.6913 - accuracy: 0.5312
Epoch 10/100
2/2 [==============================] - 0s 14ms/step - loss: 0.6730 - accuracy: 0.5312
Epoch 11/100
2/2 [=============

Epoch 84/100
2/2 [==============================] - 0s 15ms/step - loss: 0.1244 - accuracy: 0.5156
Epoch 85/100
2/2 [==============================] - 0s 15ms/step - loss: 0.1646 - accuracy: 0.5000
Epoch 86/100
2/2 [==============================] - 0s 14ms/step - loss: 0.1998 - accuracy: 0.5781
Epoch 87/100
2/2 [==============================] - 0s 14ms/step - loss: 0.0911 - accuracy: 0.3906
Epoch 88/100
2/2 [==============================] - 0s 14ms/step - loss: 0.2129 - accuracy: 0.5625
Epoch 89/100
2/2 [==============================] - 0s 14ms/step - loss: 0.1206 - accuracy: 0.3906
Epoch 90/100
2/2 [==============================] - 0s 14ms/step - loss: 0.1784 - accuracy: 0.5000
Epoch 91/100
2/2 [==============================] - 0s 15ms/step - loss: 0.2049 - accuracy: 0.3906
Epoch 92/100
2/2 [==============================] - 0s 15ms/step - loss: 0.1153 - accuracy: 0.5938
Epoch 93/100
2/2 [==============================] - 0s 14ms/step - loss: 0.2395 - accuracy: 0.4844
Epoch 94/1